In [1]:
from flask import Flask, request, jsonify
import joblib
import numpy as np
from threading import Thread

In [2]:
# Load the trained model and label encoder
model = joblib.load("hypertension_classifier_xgb.pkl")
label_encoder = joblib.load("label_encoder.pkl")  # Save this during preprocessing

In [3]:
# Initialize Flask app
app = Flask(__name__)

In [4]:
# Define a function to provide treatment recommendations
def get_treatment_recommendation(category):
    recommendations = {
        "Normal": "Maintain a healthy lifestyle with regular exercise and a balanced diet.",
        "Elevated": "Monitor your blood pressure regularly. Consider reducing salt intake and increasing physical activity.",
        "Hypertension Stage 1": "Consult a doctor. You may need lifestyle changes and possibly medication.",
        "Hypertension Stage 2": "Seek immediate medical attention. Medication and lifestyle changes are essential.",
    }
    return recommendations.get(category, "No recommendation available.")

In [5]:
# Define the prediction endpoint
@app.route("/predict", methods=["POST"])
def predict():
    try:
        # Get data from the request
        data = request.json
        systolic = data.get("systolic")
        diastolic = data.get("diastolic")
        heart_rate = data.get("heart_rate")

        # Validate input
        if not all([systolic, diastolic, heart_rate]):
            return jsonify({"error": "Missing data. Please provide systolic, diastolic, and heart rate."}), 400

        # Prepare input for the model
        input_data = np.array([[systolic, diastolic, heart_rate]])

        # Make prediction
        prediction = model.predict(input_data)
        category = label_encoder.inverse_transform(prediction)[0]

        # Get treatment recommendation
        recommendation = get_treatment_recommendation(category)

        # Return the result
        return jsonify({
            "category": category,
            "recommendation": recommendation
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [ ]:
# Function to run the Flask app in a separate thread
def run_flask():
    app.run(debug=True, use_reloader=False)

# Run Flask app in a separate thread
thread = Thread(target=run_flask)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Feb/2025 14:13:16] "GET /predict HTTP/1.1" 405 -
127.0.0.1 - - [27/Feb/2025 14:13:18] "GET /predict HTTP/1.1" 405 -
127.0.0.1 - - [27/Feb/2025 14:13:19] "GET /predict HTTP/1.1" 405 -
